Load up necessary libraries

In [ ]:
#Persistence Filter files
from libpython_persistence_filter import *
from libpython_persistence_filter_utils import *
from persistence_filter_test_utils import *

#Python scientific computing stuffs
from numpy import *
from scipy.stats import *
from matplotlib.pyplot import *
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

#Settings for rendering TeX in MATLAB
rc('text', usetex=True)

#Plotting options
fig_scaling = .5  #Scaling factor for figures
figure_size = (fig_scaling * 8.0, fig_scaling*6.0)  #Figure size, in inches
figure_folder = 'figures/'  #The folder in which to save the output

Configure simulation options

In [ ]:
#PERSISTENCE FILTER SETTINGS
#Admissible range of lambdas for use with general-purpose prior
lambda_u = 1
lambda_l = .001

#General-purpose prior log-survival time
gp_logS = lambda x : log_general_purpose_survival_function(x, lambda_l, lambda_u)

#Uniform prior log-survival time
simulation_length = 1000 #Length of simulation in seconds
uniform_logS = lambda x : asscalar(where(logical_and(0<= x, x <= simulation_length), 
                                log(subtract(1.0, (1.0 / simulation_length)*x)), 0.0))

In [ ]:
#DATA GENERATION SETTINGS

#"Standard" simulation settings
P_M_standard = .1  #Standard missed detection probability
P_F_standard = .1 #Standard false alarm probability
lambda_r_standard = 1.0 / 50.0  #Standard revisitation rate
lambda_o = 1.0  #Inter-observation rate
p_N = 1.0 / 5.0  #p_N = probability of leaving after the last reobservation; expected # observations N = 1 / p_N

num_instances = 100  #Number of Monte Carlo samples to draw for each parameter setting


#The set of times to query the belief of the persistence filter
query_times = arange(0, simulation_length, .1)

TEST EFFECTS OF THRESHOLDING ON PRECISION AND RECALL

In [ ]:
%%time

thresholds = [.01, .05, .1, .15, .2, .25, .3, .35, .4, .45, .5]

#Create lists of empty arrays that will hold valid precision/recall values
gp_persistence_filter_precisions = [array([])] * len(thresholds)
gp_persistence_filter_recalls = [array([])] * len(thresholds)

uniform_persistence_filter_precisions = [array([])] * len(thresholds)
uniform_persistence_filter_recalls = [array([])] * len(thresholds)

empirical_estimator_precisions = array([])
empirical_estimator_recalls = array([])

        
for i in range(num_instances):
    #Uniformly randomly sample a survival time
    survival_time = random.uniform(0, simulation_length)   
        
    #Sample observation times
    observation_times = sample_observation_times(lambda_r_standard, lambda_o, p_N, simulation_length)
        
    #Sample observations
    Y_binary = generate_observations(survival_time, observation_times, P_M_standard, P_F_standard)
    Y_bool = Y_binary > 0
            
    #Run the peristence filters
    gp_persistence_filter_belief = run_persistence_filter(Y_bool, observation_times, P_M_standard, P_F_standard, query_times, gp_logS)
    
    uniform_persistence_filter_belief = run_persistence_filter(Y_bool, observation_times, P_M_standard, P_F_standard, query_times, uniform_logS)
    
    #Run the empirical estimator here
    empirical_estimator_belief = run_empirical_estimator(Y_bool, observation_times, query_times)
    
    
    #Compute classifications and errors here
    X_t = query_times <= survival_time  #Ground truth
    
    #Record persistence filter precisions and recalls
    for j in range(len(thresholds)):
        #Compute the general-purpose persistence filter classification obtained by thresholding the belief state at the given threshold
        gp_persistence_filter_classifications = (gp_persistence_filter_belief >= thresholds[j])
        (P, R) = compute_feature_absence_precision_and_recall(X_t, gp_persistence_filter_classifications)
        
        #If we obtained valid precision and recall values (i.e. there were no divisions by zero due to having no predictions of the specified class)
        if(isfinite(P)):
            gp_persistence_filter_precisions[j] = append(gp_persistence_filter_precisions[j], P)
        gp_persistence_filter_recalls[j] = append(gp_persistence_filter_recalls[j], R)
        
        #Compute the uniform-prior persistence filter classification obtained by thresholding the belief state at the given threshold
        uniform_persistence_filter_classifications = (uniform_persistence_filter_belief >= thresholds[j])
        (P, R) = compute_feature_absence_precision_and_recall(X_t, uniform_persistence_filter_classifications)
        
        #If we obtained valid precision and recall values (i.e. there were no divisions by zero due to having no predictions of the specified class)
        if(isfinite(P)):
            uniform_persistence_filter_precisions[j] = append(uniform_persistence_filter_precisions[j], P)
        uniform_persistence_filter_recalls[j] = append(uniform_persistence_filter_recalls[j], R)
        

    #Record empirical estimator precision and recall
    empirical_estimator_classifications = (empirical_estimator_belief > .5)
    (P,R) = compute_feature_absence_precision_and_recall(X_t, empirical_estimator_classifications )
    
    #If we obtained valid precision and recall values (i.e. there were no divisions by zero due to having no predictions of the specified class)
    if(isfinite(P)):
        empirical_estimator_precisions = append(empirical_estimator_precisions, P)
    empirical_estimator_recalls = append(empirical_estimator_recalls, R)

#Compute the mean precision and recall for each threshold setting
gp_persistence_filter_avg_precisions = map(mean, gp_persistence_filter_precisions)
gp_persistence_filter_avg_recalls = map(mean, gp_persistence_filter_recalls)

uniform_persistence_filter_avg_precisions = map(mean, uniform_persistence_filter_precisions)
uniform_persistence_filter_avg_recalls = map(mean, uniform_persistence_filter_recalls)

#Compute the mean precision and recall for the empirical estimator
empirical_estimator_avg_precision = mean(empirical_estimator_precisions)
empirical_estimator_avg_recall = mean(empirical_estimator_recalls)


In [ ]:
#Display results
print "Uniform-prior persistence filter mean precisions:"
print uniform_persistence_filter_avg_precisions
print "Uniform-prior persistence filter mean recalls:"
print uniform_persistence_filter_avg_recalls
print "\n"

print "General-purpose persistence filter mean precisions:"
print gp_persistence_filter_avg_precisions
print "General-purpose persistence filter mean recalls:"
print gp_persistence_filter_avg_recalls
print "\n"

print "Empirical estimator filter mean precision:"
print empirical_estimator_avg_precision
print "Empirical estimator mean recall:"
print empirical_estimator_avg_recall

In [ ]:
figure(figsize=figure_size)
plot(thresholds, uniform_persistence_filter_avg_precisions, '-b', label='Uniform-prior PF') 
plot(thresholds, gp_persistence_filter_avg_precisions, '-g', label='General-purpose PF')
plot(thresholds, repeat(empirical_estimator_avg_precision, len(thresholds)), '-r', label='Empirical estimator')
#title("Precision")
xlabel("Removal threshold ($P_V$)")
ylabel("Mean precision")
legend(loc='best',frameon=False)
#savefig(figure_folder + 'persistence_estimator_precisions.pdf', bbox_inches='tight')


In [ ]:
figure(figsize=figure_size)
plot(thresholds, uniform_persistence_filter_avg_recalls, '-b', label='Uniform-prior PF') 
plot(thresholds, gp_persistence_filter_avg_recalls, '-g', label='General-purpose PF')
plot(thresholds, repeat(empirical_estimator_avg_recall, len(thresholds)), '-r', label='Empirical estimator')
#title("Recall")
xlabel("Removal threshold ($P_V$)")
ylabel("Mean recall")
legend(loc='best', frameon=False)
#savefig(figure_folder + 'persistence_estimator_recalls.pdf', bbox_inches='tight')
